# Progetto di Social Computing

a.a. 2022/2023

## Attività preliminari

### Librerie e costanti

In [ ]:
# Import delle librerie utilizzate
import os, tweepy, json
import networkx as nx 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from pyvis.network import Network


In [ ]:
# Cartelle di salvataggio
data_folder = "./data"
out_folder = "./out"
graph_folder = "./graphs"

### Funzioni

In [ ]:
# Salvataggio in locale
def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    
    with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent = 4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

In [ ]:
# Lettura da locale
def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

### Credenziali Twitter API

In [ ]:
# Caricamento credenziali da JSON
api_access = read_json("./api_access.json")

## Recupero dei follower e dei follower dei follower

### Recupero dei follower

Vogliamo recuperare, utilizzando la libreria `tweepy`, tutti i follower dell'utente *@KevinRoitero*, corredati delle seguenti informazioni:

* attributi di default;
* descrizione del profilo;
* metriche pubbliche dell'account;
* se l'account è protetto

Definiamo questi follower come *follower di primo grado*.

In [ ]:
# Inizializzamo il client
client = tweepy.Client(bearer_token=api_access["bearer_token"])

# Definiamo il nostro utente di partenza
username = "KevinRoitero"
all_user_followers = []

# Recuperiamo e memorizziamo le informazioni dell'utente
response = client.get_user(username = username, user_fields=["description", "protected", "public_metrics"])
user = dict(response.data)

# Recuperiamo e memorizziamo i follower dell'utente (con le loro info)
response = client.get_users_followers(user["id"], user_fields=["description", "protected", "public_metrics"], 
                                      max_results=150) # max_results = 150 perché i follower dell'utente sono nell'ordine di 130
for follower in response.data:
    all_user_followers.append(dict(follower))

# Associamo i follower trovati all'utente di partenza
user["followers"] = all_user_followers

# Serializziamo su JSON il risultato ottenuto
serialize_json(data_folder, "user_followers.json", user)

### Aggiunta del numero di tweet prodotti nell'ultima settimana

Ai follower trovati, si vuole aggiungere il numero di tweet pubblicati nell'ultima settimana. Per avere uniformità, troveremo anche il numero di tweet pubblicati nell'ultima settimana dall'utente *@KevinRoitero*.

In [ ]:
# Somma dei numeri di tweet prodotti in un intervallo di tempo
def sum_tweets_count(tweet_groups):
    sum = 0

    for tweet_count in tweet_groups:
        sum += tweet_count["tweet_count"]
    
    return sum

In [ ]:
# Scegliamo l'utilizzo di un client senza la possibilità di mettersi in attesa perché numero_richieste < 300, 
# dove 300 è il numero massimo di richieste per l'endpoint get_recent_tweets_count()
client = tweepy.Client(bearer_token=api_access["bearer_token"])

# Carichiamo i dati dell'utente e quelli dei suoi follower
user = read_json(data_folder+"/user_followers.json")

# Memorizziamo il numero di tweet pubblicati dall'utente nell'ultima settimana
response = client.get_recent_tweets_count(query="from:"+user["username"], granularity="day")
user["last_week_tweets_count"] = sum_tweets_count(response.data)

# Ripetiamo lo stesso per i follower dell'utente
for follower in user["followers"]:
    if(not follower["protected"]): # non si può accedere ai tweet di un utente 'protected'
        response = client.get_recent_tweets_count(query="from:"+follower["username"], granularity="day")
        follower["last_week_tweets_count"] = sum_tweets_count(response.data)

# Serializziamo su JSON il risultato ottenuto
serialize_json(data_folder, "followers_last_week_tweets.json", user)

### Recupero dei follower dei follower

Per ciascun follower di *@KevinRoitero* avente almeno 1 follower e non `protected`, vogliamo scaricare le seguenti informazioni:

* attributi di default;
* descrizione del profilo;
* metriche pubbliche dell'account;
* se l'account è protetto

Implementeremo due versioni: una in cui verranno scaricati al più 1000 follower per ogni follower di *@KevinRoitero* ed una in cui verranno invece scaricati tutti i follower dei follower.

Chiameremo questi *follower di secondo grado*. Se un follower di secondo grado è anche di primo grado, verrà da noi considerato come follower di primo grado.

#### Versione con al più mille follower di follower

Ci riferiremo da qui in poi a questa versione dei dati come ad una versione *parziale*, in quanto non presenterà lo stesso livello di completezza dell'informazione della sua controparte.

In [ ]:
# Imponiamo al client di attendere nel caso di raggiungimento del limite delle richieste
client = tweepy.Client(bearer_token=api_access["bearer_token"], wait_on_rate_limit=True)

# Carichiamo i dati dell'utente e quelli dei suoi follower
user = read_json(data_folder+"/followers_last_week_tweets.json")

# Scarichiamo al più 1000 follower per ogni follower dell'utente
for follower in user["followers"]:
    follower["followers"] = []

    # Non si possono scaricare i dettagli dei follower di profili privati
    # (e non ha senso scaricare i follower di utenti che non ne hanno)
    if (not follower["protected"] and follower["public_metrics"]["followers_count"] > 0):
        some_followers = client.get_users_followers(
            id = follower["id"], 
            user_fields = ["description", "protected", "public_metrics"],
            max_results = 1000
        )

        # Aggiungo il parse del follower ai dati finali
        for ff in some_followers.data:
            follower["followers"].append(dict(ff))

# Serializziamo il risultato
serialize_json(data_folder, "user_partial.json", user)

#### Versione con tutti i follower di follower

Prima di partire con il download, cerchiamo di farci un'idea dei volumi di dati che dovremo scaricare (e dei tempi di attesa che ne conseguono).

In [ ]:
# Carichiamo il file con le informazioni sull'utente
user = read_json(data_folder+"/followers_last_week_tweets.json")

# Se il follower non ha un profilo privato, vediamo quanti follower ha
fof_quantities = []
for follower in user["followers"]:
    if (not follower["protected"]):
        fof_quantities.append(follower["public_metrics"]["followers_count"])

# Organizziamo tutto in tabelle riassuntive
fof_df = pd.DataFrame(fof_quantities, columns=["f_count"])
print("Numero dei follower di tutti i follower")
display(fof_df.describe())

print("Numero di seguaci dei follower con più di 1000 follower")
display(fof_df.loc[fof_df["f_count"] > 1000].describe())

L'approccio che è stato seguito è quello di un download *parallelo*, ossia ciascun collaboratore al progetto si è preso in carico di scaricare una porzione di follower dei follower di *@KevinRoitero*.

In [ ]:
# Imponiamo al client di attendere nel caso di raggiungimento del limite delle richieste
client = tweepy.Client(bearer_token=api_access["bearer_token"], wait_on_rate_limit=True)

# Carichiamo i dati dell'utente e quelli dei suoi follower
user = read_json(data_folder+"/followers_last_week_tweets.json")

all_followers = user["followers"]

# Determiniamo il segmento di follower di competenza di questo client
f_num = user["public_metrics"]["followers_count"]
start = f_num*api_access["id"] // 4
end =  f_num*(api_access["id"]+1) // 4

# Memorizziamo solo i follower di secondo grado nel segmento del client
interval_followers = []

# Scarichiamo i follower di secondo grado nell'intervallo del collaboratore
for i in range(start, end):
    # Consideriamo l'i-esimo follower
    follower = all_followers[i]
    follower["followers"] = []

    # Se l'i-esimo follower non è protetto e ha dei follower, li scarichiamo tutti
    if (not follower["protected"] and follower["public_metrics"]["followers_count"] > 0):
        for all_follower_followers in tweepy.Paginator(
            client.get_users_followers,
            id = follower["id"],user_fields=["description",
            "protected", "public_metrics"],
            max_results = 1000 # chiediamo al più 1000 follower per ogni pagina di risposta
            ):

            parsed_followers = []
            
            # Interpretiamo ogni oggetto User nella pagina
            for ff in all_follower_followers.data:
                parsed_follower = {
                    "id" : ff["id"],
                    "public_metrics" : ff["public_metrics"],
                    "description" : ff["description"],
                    "name" : ff["name"],
                    "protected" : ff["protected"],
                    "username" : ff["username"]
                }
                parsed_followers.append(parsed_follower)
            
            # Aggiungiamo i dettagli dei profili interpretati all'i-esimo follower
            follower["followers"] += parsed_followers
    
    # Aggiungiamo il follower (arricchito dei suoi follower) nella lista di follower di competenza del collaboratore
    interval_followers.append(follower)

# Salviamo la porzione di follower scaricata
serialize_json(data_folder, "f_of_f_"+str(api_access["id"])+".json", interval_followers)

Si uniscono ora i file generati dai diversi collaboratori in un unico file JSON.

In [ ]:
# Carichiamo il JSON con le informazioni su @KevinRoitero
user = read_json(data_folder+"/followers_last_week_tweets.json")
followers = []

for i in range(4):
    # Carichiamo i follower individuati dal collaboratore i
    partial_followers = read_json(data_folder+"/f_of_f_"+str(i)+".json")
    
    # Carichiamo all'elenco totale dei follower
    followers += partial_followers

# Aggiorniamo i dati sui follower di @KevinRoitero
user["followers"] = followers

# Serializziamo il risultato ottenuto
serialize_json(data_folder, "user_full.json", user)

### Riformattazione del JSON finale

Al fine di essere conformi alle specifiche della consegna, si decide di ristrutturare (ossia modificarne la presentazione, pur mantenendo invariate le informazioni al suo interno) i JSON `user_partial.json` e `user_full.json` definiti in precedenza.

Il formato che è stato ritenuto come più opportuno, sia per un'efficienza di archiviazione che di computazione (importante per le sezioni successive), è il seguente:
```json
{
    ...
    id (int) : {
        "name" : str,
        "username" : str,
        "description" : int,
        "public_metrics" : {
            "followers_count" : int,
            "following_count" : int,
            "tweet_count" : int,
            "listed_count" : int
        },
        "protected" : bool,
        ("last_week_tweets_count" :int,)
        ("followers" : [
            ...
            f_id (int),
            ...
        ])
    }
    ...
}
```

Dove `id` rappresenta l'identificativo univoco di un utente, le cui informazioni sono state scaricate nei passi precedenti, ed `f_id` rappresenta l'identificativo di un follower di `id`.

È doveroso far notare che, per come sono stati scaricati gli utenti, solamente coloro che o sono *@KevinRoitero* o sono suoi follower (non `protected`) presentano il campo `last_week_tweets_count` ed il campo `followers` (da cui le parentesi tonde nello schema).

Chiameremo il formato prodotto dai download *formato ad albero* ed il formato prodotto da questa conversione *formato a tabella hash*.

In [ ]:
# Definiamo una funzione che esegue una normalizzazione di un singolo utente
def normalize_into(user, root_distance, destination):
    # Normalizziamo l'utente considerato
    normalized = {
        "name" : user["name"],
        "username" : user["username"],
        "description" : user["description"],
        "public_metrics" : user["public_metrics"],
        "protected" : user["protected"]
    }

    # Se user non è un follower di secondo grado e non è protetto, si possono definire i dettagli aggiuntivi
    if (root_distance < 2 and not user["protected"]):
        # Estraiamo le ids dei follower dell'utente
        f_ids = []
        for f in user["followers"]:
            f_ids.append(f["id"])

        # Normalizziamo le informazioni restanti
        normalized["last_week_tweets_count"] = user["last_week_tweets_count"]
        normalized["followers"] = f_ids
    
    # Se l'utente non è già definito in 'destination', lo aggiungiamo
    if user["id"] not in destination:
        destination[user["id"]] = normalized

In [ ]:
# Dato un json strutturato ad albero, lo trasforma in un json a tabella hash
def normalize_json(path, final_name):
    # Carichiamo il JSON totale, non formattato
    root_user = read_json(path)

    nodes = {} # JSON finale

    # Normalizziamo @KevinRoitero
    normalize_into(user = root_user, root_distance=0, destination = nodes)

    # Facciamo la stessa cosa per follower i suoi follower
    for follower in root_user["followers"]:
        normalize_into(user = follower, root_distance = 1, destination = nodes)

    # ... e per i follower dei follower
    for follower in root_user["followers"]:
        if (not follower["protected"]):
            for ff in follower["followers"]:
                normalize_into(user = ff, root_distance = 2, destination = nodes)

    # Serializziamo il JSON ben formattato
    serialize_json(data_folder, final_name+".json", nodes)


In [ ]:
# Riformattazione JSON parziale
normalize_json(path=data_folder+"/user_partial.json", final_name="some_users_final")

# Riformattazione JSON totale
normalize_json(path=data_folder+"/user_full.json", final_name="all_users_final")

Consideriamo che, per come è stato costruito, i file `*_users_final.json` presenta una struttura ordinata:

1. Al primo posto vi sonoi dettagli sul profilo di *@KevinRoitero*;
2. dal secondo posto fino al 134-esimo vi sono i follower di *@KevinRoitero*;
3. dal 135-esimo posto in poi vi sono i follower dei follower di *@KevinRoitero*.

## Generazione dei grafi

### Generazione del grafo della rete sociale diretta

Vogliamo generare un grafo che abbia come nodi *@KevinRoitero* ed i suoi follower, ognuno di questi deve rispettare le seguenti caratteristiche:

* il suo `id` deve essere uguale all'`id` del profilo utente;
* deve avere come attributi:
    * lo username;
    * la descrizione;
    * il numero di follower del profilo.

Infine, vogliamo aggiungere archi $(v,w)$ tra due nodi $v$ e $w$ se e solo se il profilo corrispondente a $v$ è follower del profilo corrispondente a $w$.

In [ ]:
# Crea un grafo diretto dove un arco tra nodi indica una relazione di "following"
def create_following_graph(users, name):
    # Creiamo un grafo diretto vuoto
    following_graph = nx.DiGraph()

    # Inseriamo @KevinRoitero
    main_id = list(users.keys())[0]
    main = users[main_id]
    following_graph.add_node(
        int(main_id),
        username = main["username"],
        description = main["description"],
        followers_count = main["public_metrics"]["followers_count"]
    )

    # Inseriamo i follower
    for id in main["followers"]:
        user = users[str(id)]
        following_graph.add_node(
            id,
            username = user["username"], 
            description = user["description"],
            followers_count = user["public_metrics"]["followers_count"]
        )

    # Aggiungiamo gli archi tra i nodi
    for w in following_graph.nodes:
        w_key = str(w)
        # Sappiamo le relazioni di following solo per gli utenti con profilo pubblico
        if not users[w_key]["protected"]:
            for v in users[w_key]["followers"]:
                if (v in following_graph.nodes):
                    following_graph.add_edge(v, w)

    # Serializziamo il grafo finale
    serialize_json(graph_folder, name+"_following_graph.json", nx.node_link_data(following_graph))

    return following_graph

In [ ]:
# Carichiamo i due json a tabella di hash
partial_users = read_json(data_folder+"/some_users_final.json")
full_users = read_json(data_folder+"/all_users_final.json")

# Grafo parziale
partial_fg = create_following_graph(partial_users, "some_users")
# Grafo totale
full_fg = create_following_graph(full_users, "all_users")

### Generazione grafo con preferential attachment 

Generiamo un secondo grafo a partire da quello definito in precedenza, nel seguente modo:

1. convertiamo `*_following_graph` in grafo indiretto;
2. a tale grafo indiretto aggiungiamo dei nodi, utilizzando il metodo del preferential attachment, tali che:
    - il numero di nodi aggiunti è uguale al numero di nodi già presenti nel grafo;
    - ogni nodo aggiunto ha due archi uscenti.

Non essendo specificati ulteriori dettagli nella consegna, ci riconduciamo al modello più semplice utilizzando un attaccamento preferenziale lineare.

In [ ]:
def convert_to_preferential_graph(directed_graph, name):
    # Creiamo il grafo indiretto intermedio
    number_of_nodes = directed_graph.number_of_nodes()
    undirected_graph = directed_graph.to_undirected()

    # Aggiungiamo i nodi utilizzando il preferential attachment lineare
    preferential_graph = nx.barabasi_albert_graph(number_of_nodes*2, 2, initial_graph = undirected_graph)

    # Serializzazione del grafo
    serialize_json(graph_folder, name+"_preferential_graph.json", nx.node_link_data(preferential_graph))
    return preferential_graph

In [ ]:
# Carichiamo i grafi definiti in precedenza
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))

# Grafo "preferenziale" parziale
partial_pg = convert_to_preferential_graph(partial_fg, "some_users")
# Grafo "preferenziale" totale
full_pg = convert_to_preferential_graph(full_fg, "all_users")

## Analisi dei grafi

### Visualizzazione interattiva dei due grafi

Utilizzando la libreria `pyivs`, verrà creata una visualizzazione interattiva del grafo dei follower e del grafo "prefernziale" (`preferential_graph`).

In [ ]:
# Definiamo una funzione per visualizzare interattivamente un grafo
def show_graph(graph, graph_name):
    # Rappresentiamo gli archi come frecce se il grafo è diretto
    if (nx.is_directed(graph)):
        network = Network(height="720px", directed=True, bgcolor="#222222", font_color="white") # width = 100% default
    else:
        network = Network(height="720px", bgcolor="#222222", font_color="white") # width = 100% default
    network.barnes_hut() # layout
    network.from_nx(graph)

    # Aggiungiamo le etichette ai nodi
    neighbours = network.get_adj_list()
    for node in network.nodes:
        node["value"] = len(neighbours[node["id"]])
    
    # Mostriamo la visualizzazione interattiva
    network.show(out_folder+"/"+graph_name+".html")

In [ ]:
# Carichiamo i due grafi diretti
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))

# Visualizziamo i grafi
show_graph(partial_fg, "partial_fg")
show_graph(full_fg, "full_fg")

In [ ]:
# Carichiamo i due grafi indiretti
partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))

# Visualizziamo i grafi
show_graph(partial_pg, "partial_pg")
show_graph(full_pg, "full_pg")

### Visualizzazione dei grafi attraverso nodi di dimensione variabile

Si vogliono disegnare i due grafi definiti nelle sezioni precedenti in modo che la dimensione dei nodi raffigurati sia direttamente proporzionale:

- all'in-degree (grado in entrata) per il grafo diretto
- al degree (grado) per il grafo indiretto

 Si userà l'algoritmo di [Fruchterman-Reingold](https://github.com/gephi/gephi/wiki/Fruchterman-Reingold) per la distribuzione dei nodi.

In [ ]:
# Disegna un grafo in cui la dimensione dei nodi è direttamente proporzionale all'(in-)degree
def draw_graph(graph):
    node_size = []
    k = 4 # costante di "ingrandimento" dei nodi
    # A seconda della natura del grafo, scegliamo quale metrica usare
    if nx.is_directed(graph):
        for node in graph.nodes():
            node_size.append(graph.in_degree(node)*k)
    else:
        for node in graph.nodes():
            node_size.append(graph.degree(node)*k)

    # Disegnamo il grafo
    plot = nx.draw_networkx(
        graph, 
        pos = nx.spring_layout(graph), # algoritmo di Fruchterman-Reingold
        node_color = '#A0CBE2',
        edge_color = (0,0,0,0.1), # gli archi sono trasparenti al fine di far risaltare i nodi
        with_labels = False,
        node_size = node_size
    )

    return plot

# Mette a confronto due grafi i cui nodi hanno dimensione proporzionale al grado
def compare_degrees(partial_G, total_G, name):
    plt.subplots(figsize=(10, 10))

    # Disegnamo il grafo parziale
    plt.subplot(2, 1, 1)
    draw_graph(partial_G)
    plt.title("Grafo parziale")

    # Disegnamo il grafo totale
    plt.subplot(2, 1, 2)
    draw_graph(total_G)
    plt.title("Grafo totale")

    # Salviamo e mostriamo quanto disegnato
    plt.savefig(out_folder+"/"+name+".pdf")
    plt.show()

In [ ]:
# Carichiamo i due grafi diretti
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))

# Disegnamo i due grafi
compare_degrees(partial_fg, full_fg, "following_graphs")

In [ ]:
# Si caricano i due grafi indiretti
partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))

# Disegnamo i due grafi
compare_degrees(partial_pg, full_pg, "preferential_graphs")

Ci interessa visualizzare in maniera più chiara i gradi dei grafi: disegnamo quindi la distribuzione dei gradi. 

In [ ]:
# Restituisce una lista di frequenze (non razionali, intere)
def degree_histogram_directed(G, in_degree=False, out_degree=False):
    nodes = G.nodes()
    # Popoliamo la lista con tutti i (in/out-)gradi dei nodi
    if in_degree:
        in_degree = dict(G.in_degree())
        degseq=[in_degree.get(k,0) for k in nodes]
    elif out_degree:
        out_degree = dict(G.out_degree())
        degseq=[out_degree.get(k,0) for k in nodes]
    else:
        degseq=[v for k, v in G.degree()]

    dmax=max(degseq)+1
    # Creiamo una lista di frequenza dei gradi
    freq= [ 0 for d in range(dmax) ]
    for d in degseq:
        freq[d] += 1
    # Retituiamo la lista di frequenza, ordinata crescentemente
    return freq

# Disegna il confronto della distribuzione dei gradi
def compare_degree_distribution(partial_G, total_G, in_degree=False, out_degree=False):
    if (nx.is_directed(partial_G)):
        # Disegniamo la distribuzione dell'in/out-degree per il grafo diretto
        degree_freq_partial = degree_histogram_directed(partial_G, in_degree=in_degree, out_degree=out_degree)
        degree_freq_total = degree_histogram_directed(total_G, in_degree=in_degree, out_degree=out_degree)
        plt.loglog(range(len(degree_freq_partial)), degree_freq_partial, 'go', label='parziale')
        plot = plt.loglog(range(len(degree_freq_total)), degree_freq_total, 'r^', label='totale')
    else:
        # Disegniamo la distribuzione dell'in/out-degree per il grafo indiretto
        degree_freq_partial = nx.degree_histogram(partial_G)
        degree_freq_total = nx.degree_histogram(total_G)
        plt.loglog(range(len(degree_freq_partial)), degree_freq_partial,'go', label="parziale")
        plot = plt.loglog(range(len(degree_freq_total)), degree_freq_total,'r^', label="totale")
    
    plt.legend()
    plt.xlabel('Grado')
    plt.ylabel('Numero di nodi')
    # Restituiamo il disegno della distribuzione
    return plot

In [ ]:
# Carichiamo i grafi
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))
partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))

# Fissiamo la dimensione dei grafici
fig, ax = plt.subplots(figsize=(15,5))

# Mostriamo la distribuzione degli in-degree
plt.subplot(1, 3, 1)
compare_degree_distribution(partial_fg, full_fg, in_degree=True)
plt.title("Distribuzione degli in-degree")

# Mostriamo la distribuzione degli out-degree
plt.subplot(1, 3, 2)
compare_degree_distribution(partial_fg, full_fg, out_degree=True)
plt.title("Distribuzione degli out-degree")

# Mostriamo la distribuzione dei gradi
plt.subplot(1, 3, 3)
compare_degree_distribution(partial_pg, full_pg)
plt.title("Distribuzione dei gradi")

plt.suptitle("Distribzione dei gradi nei grafi diretti")
plt.savefig(out_folder+"/degree_distributions.pdf")


### Visualizzazione della più grande componente fortemente connessa

Per ciascuno dei due grafi, si produce una visualizzazione statica del grafo con una colorazione rossa dei nodi appartenenti alla più grande componente fortemente connessa (componente gigante), nera per gli altri.

In [ ]:
# Restituisce il sottografo con la componente gigante
def get_giant_component_subgraph(graph):
    # Identifichiamo i nodi appartenenti alla componente gigante
    nodes = []
    if nx.is_directed(graph):
        nodes = max(nx.strongly_connected_components(graph), key=len)
    else:
        # Non ha senso parlare di CFC in un grafo diretto, possiamo cercare la componente connessa
        nodes = max(nx.connected_components(graph), key=len)
    
    # Restituiamo il sottografo corrispondente
    return nx.subgraph(graph, nodes)

# Disegna 'graph' evidenziando in rosso i nodi della componente gigante
def highlight_giant_component(graph):
    # Identifichiamo il sottografo della componente gigante
    giant = get_giant_component_subgraph(graph)

    # Mappiamo a ciascun nodo di 'graph' il colore appropriato
    color_map=[]
    for node in graph:
        if node in giant.nodes:
            # Se il nodo fa parte della SCC più grande viene colorato di rosso
            color_map.append('red')
        else:
            # Altrimenti di nero
            color_map.append('black')

    # Disegnamo il grafo con la mappatura definita
    plot = nx.draw_networkx(
        graph,
        pos = nx.spring_layout(graph), # algoritmo di Fruchterman-Reingold
        node_size = 100,
        node_color = color_map,
        edge_color = (0,0,0,0.3), # gli archi sono trasparenti al fine di far risaltare i nodi
        with_labels = False
    )
    
    return plot

# Mostra un'immagine di confronto tra due componenti giganti di due grafi
def compare_giant_components(partial_G, total_G, name):
    # Ci assicuriamo che le immagini siano sufficientemente grandi
    plt.subplots(figsize=(12,6))

    # Disegnamo il grafo parziale
    plt.subplot(1, 2, 1)
    highlight_giant_component(partial_G)
    plt.title("Grafo parziale")

    # Disegnamo il grafo totale
    plt.subplot(1, 2, 2)
    highlight_giant_component(total_G)
    plt.title("Grafo totale")

    # Salviamo e mostriamo quanto disegnato
    plt.savefig(out_folder+"/"+name+".pdf")
    plt.show()


In [ ]:
# Carichiamo i grafi diretti
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))

# Confrontiamo le componenti giganti dei due grafi
compare_giant_components(partial_fg, full_fg, "following_giant_component")

Si vuole capire quanto sono grandi le due componenti giganti.

In [ ]:
# Carichiamo i due grafi diretti
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))

# Calcoliamo la grandezza (in nodi) della componente gigante
dim_partial = len(list(get_giant_component_subgraph(partial_fg).nodes()))
dim_full = len(list(get_giant_component_subgraph(full_fg).nodes()))

# Mostriamo i valori
print(f"\nDimensione SCC del grafo parziale: {dim_partial}\nDimensione SCC del grafo totale: {dim_full}")

In [ ]:
# Carichiamo i grafi indiretti
partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))

# Confrontiamo le componenti giganti dei due grafi
compare_giant_components(partial_pg, full_pg, "preferential_giant_component")

### Distanze

Vogliamo calcolare le seguenti distanze sui due grafi:

- il centro;
- il raggio;
- la distanza media;
- la distanza massima (diametro).

Il centro, il raggio e la distanza massima (diametro) dipendono dal valore dell'eccentricità: se questa è infinita, non è molto informativa sul grafo. Risolviamo questo problema sostituendo al grafo la componente gigante per queste metriche.

In [ ]:
# Importamo i grafi rilevanti
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))
partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))

# Computiamo le massime scc per i grafi diretti
partial_scc = get_giant_component_subgraph(partial_fg)
full_scc = get_giant_component_subgraph(full_fg)

# Prepariamo un dizionario delle distanze
distances = {}

#centro
distances["center"] = [
    list(nx.center(partial_scc)),
    list(nx.center(full_scc)),
    list(nx.center(partial_pg)),
    list(nx.center(full_pg))
]
#raggio
distances["radius"] = [
    nx.radius(partial_scc),
    nx.radius(full_scc),
    nx.radius(partial_pg),
    nx.radius(full_pg)
]
#distanza media
distances["avarage_distance"] = [
    nx.average_shortest_path_length(partial_fg),
    nx.average_shortest_path_length(full_fg),
    nx.average_shortest_path_length(partial_pg),
    nx.average_shortest_path_length(full_pg)
]
#distanza massima 
distances["max_distance"] = [
    nx.diameter(partial_scc),
    nx.diameter(full_scc),
    nx.diameter(partial_pg),
    nx.diameter(full_pg)
]

# Visualizziamo le distanze attraverso una tabella
display(pd.DataFrame(distances, index=["partial_fg", "full_fg", "partial_pg", "full_pg"]))
serialize_json(data_folder, "distances.json", distances)

Ci interessa capire quanti sono i centri di ogni grafo/componente gigante.

In [ ]:
# Carico i dati sulle distanze
distances = read_json(data_folder+"/distances.json")
centers = distances["center"]

# Stampo il numero di centri di ogni grafo
for i in range(4):
    print(len(centers[i]))

### Calcolo delle misure di centralità

Vogliamo calcolare le seguenti metriche inerenti alla centralità dei due grafi:

- centralità di betweenness;
- centralità di prossimità;
- centralità di grado (in entrata/uscita);
- PageRank;
- HITS.

In [ ]:
# Restituisce la lista di valori di un dizionario
def to_list(dict):
    return list(dict.values())

# Compone un dizionario con le metriche di centralità rilevanti
def compute_centrality_metrics(graph):
    metrics = {}
    # Betweenness
    metrics["betweenness"] = to_list(nx.betweenness_centrality(graph))
    # Closeness
    metrics["closeness"] = to_list(nx.closeness_centrality(graph))
    # (in/out-degree)
    if(nx.is_directed(graph)):
        metrics["in_degree"] = to_list(nx.in_degree_centrality(graph))
        metrics["out_degree"] = to_list(nx.out_degree_centrality(graph))
    # degree
    metrics["degree"] = to_list(nx.degree_centrality(graph))
    # PageRank
    metrics["pagerank"] = to_list(nx.pagerank(graph))
    # HITS
    metrics["hits_hubs"] = to_list(nx.hits(graph)[0])
    metrics["hits_authorities"] = to_list(nx.hits(graph)[1])

    return metrics

In [ ]:
# Importiamo i quattro grafi
partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))
partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))

# Calcoliamo le metriche di centralità per i quattro grafi
partial_fg_centrality = compute_centrality_metrics(partial_fg)
full_fg_centrality = compute_centrality_metrics(full_fg)
partial_pg_centrality = compute_centrality_metrics(partial_pg)
full_pg_centrality = compute_centrality_metrics(full_pg)

# Mostriamo le metriche calcolate sotto forma di tabella
print("Grafo diretto parziale")
display(pd.DataFrame(partial_fg_centrality, index=partial_fg.nodes()))
print("Grafo diretto totale")
display(pd.DataFrame(full_fg_centrality, index=full_fg.nodes()))
print("Grafo \"preferenziale\" parziale")
display(pd.DataFrame(partial_pg_centrality, index=partial_pg.nodes()))
print("Grafo \"preferenziale\" totale")
display(pd.DataFrame(full_pg_centrality, index=full_pg.nodes()))

# Salviamo i dati
serialize_json(data_folder, "partial_fg_centrality.json", partial_fg_centrality)
serialize_json(data_folder, "full_fg_centrality.json", full_fg_centrality)
serialize_json(data_folder, "partial_pg_centrality.json", partial_pg_centrality)
serialize_json(data_folder, "full_pg_centrality.json", full_pg_centrality)

Riteniamo sia più comodo riassumere queste informazioni in modo da poter riuscire ad eseguire osservazioni sensate.

In [ ]:
print("Grafo diretto parziale")
display(pd.DataFrame(partial_fg_centrality, index=partial_fg.nodes()).describe())
print("Grafo diretto totale")
display(pd.DataFrame(full_fg_centrality, index=full_fg.nodes()).describe())
print("Grafo \"preferenziale\" parziale")
display(pd.DataFrame(partial_pg_centrality, index=partial_pg.nodes()).describe())
print("Grafo \"preferenziale\" totale")
display(pd.DataFrame(full_pg_centrality, index=full_pg.nodes()).describe())

Anche se sono state riassunte, questi dati comunicano ancora troppo poco. Riteniamo sia più utile visualizzare graficamente queste informazioni. 

In [ ]:
# Disegna il grafo evvidenziando il valore di una misura con heatmap
def draw_metric(G, pos, measures, measure_name):
    # Disegnamo i nodi colorati mediante heatmap coerentemente al valore di 'measures'
    nodes = nx.draw_networkx_nodes(G, pos, node_size=50, cmap=plt.cm.plasma, 
                                   node_color=measures)
    nx.draw_networkx_edges(G, pos, edge_color=(0,0,0,0.1)) # gli archi non sono essenziali
    plt.title(measure_name)
    plt.colorbar(nodes) # legenda della heatmap
    plt.axis('off')
    return nodes

# Compara una metrica di centralità dei 4 grafi in esame
def draw_metric_comparison(centrality_metric):
    # Carichiamo i grafi con le rispettive metriche di centralità
    partial_fg = nx.node_link_graph(read_json(graph_folder+"/some_users_following_graph.json"))
    partial_fg_centrality = read_json(data_folder+"/partial_fg_centrality.json")
    full_fg = nx.node_link_graph(read_json(graph_folder+"/all_users_following_graph.json"))
    full_fg_centrality = read_json(data_folder+"/full_fg_centrality.json")
    partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
    partial_pg_centrality = read_json(data_folder+"/partial_pg_centrality.json")
    full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))
    full_pg_centrality = read_json(data_folder+"/full_pg_centrality.json")

    # Fissiamo la dimensione dei grafi
    plt.subplots(figsize=(10,10))

    # Si mostra il grafo diretto parziale
    plt.subplot(2, 2, 1)
    draw_metric(
        partial_fg, 
        nx.spring_layout(partial_fg), 
        partial_fg_centrality[centrality_metric], 
        'Grafo diretto parziale'
    )

    # Si mostra il grafo diretto totale
    plt.subplot(2, 2, 2)
    draw_metric(
        full_fg, 
        nx.spring_layout(full_fg), 
        full_fg_centrality[centrality_metric], 
        'Grafo diretto totale'
    )

    # Si mostra il grafo indiretto parziale
    plt.subplot(2, 2, 3)
    draw_metric(
        partial_pg, 
        nx.spring_layout(partial_pg), 
        partial_pg_centrality[centrality_metric], 
        'Grafo indiretto parziale'
    )

    # Si mostra il grafo indiretto totale
    plt.subplot(2, 2, 4)
    draw_metric(
        full_pg, 
        nx.spring_layout(full_pg), 
        full_pg_centrality[centrality_metric], 
        'Grafo indiretto totale'
    )

    plt.suptitle(centrality_metric.capitalize()+" Centrality")
    
    # Salviamo e mostriamo il grafo
    plt.savefig(out_folder+"/"+centrality_metric+".pdf")

In [ ]:
# Betweenness
draw_metric_comparison("betweenness")

In [ ]:
# Closeness
draw_metric_comparison("closeness")

In [ ]:
# Grado
draw_metric_comparison("degree")

In [ ]:
# PageRank
draw_metric_comparison("pagerank")

In [ ]:
# Hub di HITS
draw_metric_comparison("hits_hubs")

In [ ]:
# Authorities di HITS
draw_metric_comparison("hits_authorities")

### Coefficenti per la stima della "small-worldness" 

I coefficienti per la "small-worldness" del grafo indiretto sono:

- il coefficiente omega;
- il coefficiente sigma.

Li calcoliamo solo sui grafi indiretti perché ha poco senso parlare di reti piccolo-mondo in un grafo diretto (è più difficile realizzare l'ipotesi di grafo connesso).

In [ ]:
# Carichiamo i grafi indiretti
partial_pg = nx.node_link_graph(read_json(graph_folder+"/some_users_preferential_graph.json"))
full_pg = nx.node_link_graph(read_json(graph_folder+"/all_users_preferential_graph.json"))

# Calcoliamo i valori
small_worldness = {}
small_worldness["omega"] = [nx.omega(partial_pg), nx.omega(full_pg)]
small_worldness["sigma"] = [nx.sigma(partial_pg), nx.sigma(full_pg)]

# Mostriamo i risultati in tabella
display(pd.DataFrame(small_worldness, index=["partial_pg", "full_pg"]))
serialize_json(data_folder, "small_worldness.json", small_worldness)